In [173]:
from enum import Enum

import numpy as np
import pyvista as pv
import trimesh as tm
import vedo as vd

vd.settings.set_vtk_verbosity(0)

# Load Dataset

In [174]:
patient_id = "patient_01"
infile_name = f"../data/raw/{patient_id}_mesh_with_uacs_fibers_tags.vtk"

vtk_mesh = pv.read(infile_name)
pv_mesh = pv.PolyData.from_regular_faces(
    vtk_mesh.points, vtk_mesh.cells.reshape(-1, 4)[:, 1:]
)

# Clean Up Anatomical Regions

In [175]:
class LegacyAnatomicalTags(Enum):
    BODY = 11
    LAA = 13
    LIPV = 21
    LSPV = 23
    RIPV = 25
    RSPV = 27


class AnatomicalTags(Enum):
    BODY = 0
    LAA = 1
    LIPV = 2
    LSPV = 3
    RIPV = 4
    RSPV = 5

### Deterministic Fixes, Smoothing, Clipping

In [176]:
region_tag = LegacyAnatomicalTags.RSPV.value
new_tag = 1

pv_region_mesh = vtk_mesh.extract_values(region_tag, scalars="anatomical_tags")
tm_region_mesh = tm.Trimesh(
    pv_region_mesh.points, pv_region_mesh.cells.reshape(-1, 4)[:, 1:], process=False
)
tm_region_mesh.fill_holes()
connected_segments = tm_region_mesh.split(only_watertight=False)
main_segment = connected_segments[0]

segment_cell_inds = pv_mesh.find_containing_cell(main_segment.triangles_center)
new_anatomical_tags = np.zeros(pv_mesh.n_cells)
new_anatomical_tags[segment_cell_inds] = new_tag
pv_mesh.cell_data["anatomical_tags"] = new_anatomical_tags
pv_mesh = pv_mesh.cell_data_to_point_data()

vd_mesh = vd.Mesh([pv_mesh.points, pv_mesh.faces.reshape(-1, 4)[:, 1:]])
vd_mesh.pointdata["anatomical_tags"] = pv_mesh.point_data["anatomical_tags"]
vd_mesh = vd_mesh.smooth_data(niter=50)
pv_mesh.point_data["anatomical_tags"] = vd_mesh.pointdata["anatomical_tags"]
pv_region_mesh = pv_mesh.extract_values(ranges=[0.5, 1.1], scalars="anatomical_tags")

In [177]:
plotter = pv.Plotter(window_size=[900, 900])
plotter.add_mesh(
    pv_mesh,
    style="wireframe",
    color="lightgrey",
)
plotter.add_mesh(
    pv_region_mesh,
    color="red",
)
plotter.show()

Widget(value='<iframe src="http://localhost:46043/index.html?ui=P_0x7efd75b82c10_43&reconnect=auto" class="pyv…